In [1]:
'''
We will need the stopwords from NLTK and spacy’s en model for text pre-processing.
Later, we will be using the spacy model for lemmatization.

Lemmatization is nothing but converting a word to its root word.
For example: the lemma of the word ‘machines’ is ‘machine’.

Likewise, ‘walking’ –> ‘walk’, ‘mice’ –> ‘mouse’ and so on.

'''

# Run in python console
import nltk
nltk.download('stopwords')

# nltk.download('stopwords') # You already did this in a previous lab

[nltk_data] Downloading package stopwords to C:\Users\Yong
[nltk_data]     Jie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim (Topic Modeling Pacakge)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# spacy for lemmatization
import spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

C:\Anaconda\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [6]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


In [8]:
'''
We will be using the 20-Newsgroups dataset for this exercise.
This version of the dataset contains about 11k newsgroups posts from 
20 different topics. This is available as newsgroups.json.
'''
# Import Dataset
df = pd.read_csv('./greyandsanders.csv', encoding='utf-8')

df.head()

,date,username,text
0,2017-03-03 07:36:59,michelleyuan,💛
1,2018-10-11 01:59:34,mdflmvmt,"Hi, come meditate with us! 🧘🏻‍♀️.."
2,2017-05-01 17:58:34,shuchsmooch,When are u going @theprojectweekend !!!!! Can ...
3,2017-04-30 18:15:10,theprojectweekend,@shuchsmooch I'm going down again next week! I...
4,2017-04-30 17:24:30,shuchsmooch,Omg I love this !!!!!


In [9]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())





[nltk_data] Downloading package words to C:\Users\Yong
[nltk_data]     Jie\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [10]:
'''
As you can see there are many emails, newline and extra spaces that is quite distracting.
Let’s get rid of them using regular expressions.
'''
# Convert to list
data = df.text.values.tolist()
new_data = []

# Remove Emails
data = [re.sub(r'[^\x00-\x7F]+', '', sent) for sent in data]

# # Remove new line characters
# data = [re.sub('\s+', ' ', sent) for sent in data]

# # Remove distracting single quotes
# data = [re.sub("\'", "", sent) for sent in data]

        
    

print(data[:20])

['', 'Hi, come meditate with us! ..', 'When are u going @theprojectweekend !!!!! Can I tag along??????? I wanna check out ', "@shuchsmooch I'm going down again next week! I've got my eye on a couple of chairs and a side table. Can customise the pieces (type of wood, colour, size)! Check it out.", 'Omg I love this !!!!!', '#greyandsanders #pickjunction #furniture #design #home #interiors #interiordesign #decor #sgdesign #singapore #sgig #igsg #instasg #homedecor #home', 'The best of the best', '', 'Hi, how much is this for a 2m table?', '', 'Hi @mynameisiska! Thanks for your interest (: this table is indeed an Oak piece. We do have some Oak slabs on display in our studio. Feel free to drop by to take a look!', 'Hi there! What wood is this? Oak I reckon? Any of such slabs in stock? ', 'Looks like the one at Noshery too!', 'Thank you for for the lovely work !! We love the end product too !!', 'Stunning wall!', 'Wow! That wall!! ', '@jrwatts123 wallpaper ', 'Love this wall xo', '@delyanaku

In [11]:
'''
After removing the emails and extra spaces, the text still looks messy.
It is not ready for the LDA to consume. You need to break down each sentence
into a list of words through tokenization, while clearing up all the messy 
text in the process.

Let’s tokenize each sentence into a list of words, removing punctuations and 
unnecessary characters altogether.

Gensim’s simple_preprocess() is great for this.
Additionally I have set deacc=True to remove the punctuations.
'''
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[0])
for list_of_words in data_words:
    for word in list_of_words:
        if word.lower() in words or not word.isalpha():
            list_of_words.remove(word)
    
print(data_words[:80])

[]
[[], ['come', 'with'], ['are', 'can', 'along', 'wanna', 'out'], ['shuchsmooch', 'down', 'next', 've', 'my', 'on', 'of', 'chairs', 'side', 'can', 'customise', 'pieces', 'of', 'colour', 'check', 'out'], ['omg', 'this'], ['greyandsanders', 'pickjunction', 'design', 'interiors', 'interiordesign', 'decor', 'sgdesign', 'singapore', 'sgig', 'igsg', 'instasg', 'homedecor'], ['the', 'best'], [], ['how', 'is', 'for'], [], ['mynameisiska', 'for', 'interest', 'table', 'indeed', 'oak', 'we', 'have', 'oak', 'slabs', 'display', 'our', 'feel', 'by', 'to', 'take'], ['there', 'wood', 'this', 'reckon', 'of', 'slabs', 'stock'], ['looks', 'the', 'at', 'noshery'], ['you', 'for', 'lovely', 'we', 'the', 'product'], ['wall'], ['that'], ['jrwatts'], ['this', 'xo'], ['delyanakumaat'], ['senkee'], [], [], ['decor', 'inspirations', 'your', 'is', 'let', 'know', 'you', 'any', 'plans', 'mind'], ['huat', 'let', 'know', 'you', 'looking', 'really', 'tiles'], ['id', 'to', 'where', 'the', 'walnut', 'slabs', 'sourced'],

In [12]:
data_words = [x for x in data_words if x]

print(data_words[0:20])

[['come', 'with'], ['are', 'can', 'along', 'wanna', 'out'], ['shuchsmooch', 'down', 'next', 've', 'my', 'on', 'of', 'chairs', 'side', 'can', 'customise', 'pieces', 'of', 'colour', 'check', 'out'], ['omg', 'this'], ['greyandsanders', 'pickjunction', 'design', 'interiors', 'interiordesign', 'decor', 'sgdesign', 'singapore', 'sgig', 'igsg', 'instasg', 'homedecor'], ['the', 'best'], ['how', 'is', 'for'], ['mynameisiska', 'for', 'interest', 'table', 'indeed', 'oak', 'we', 'have', 'oak', 'slabs', 'display', 'our', 'feel', 'by', 'to', 'take'], ['there', 'wood', 'this', 'reckon', 'of', 'slabs', 'stock'], ['looks', 'the', 'at', 'noshery'], ['you', 'for', 'lovely', 'we', 'the', 'product'], ['wall'], ['that'], ['jrwatts'], ['this', 'xo'], ['delyanakumaat'], ['senkee'], ['decor', 'inspirations', 'your', 'is', 'let', 'know', 'you', 'any', 'plans', 'mind'], ['huat', 'let', 'know', 'you', 'looking', 'really', 'tiles'], ['id', 'to', 'where', 'the', 'walnut', 'slabs', 'sourced']]


In [13]:
'''
Bigrams are two words frequently occurring together in the document.
Trigrams are 3 words frequently occurring.

Some examples in our example are: ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.

Gensim’s Phrases model can build and implement the bigrams, trigrams, 
quadgrams and more.

The two important arguments to Phrases are min_count and threshold.
The higher the values of these param, the harder it is for words to be combined to bigrams.
'''
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['come', 'with']


In [14]:
'''
The bigrams model is ready. Let’s define the functions to remove the stopwords, 
make bigrams and lemmatization and call them sequentially.
'''
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
'''
Let’s call the functions in order.
'''
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['come']]


In [16]:
'''
The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus.
Let’s create them.
'''
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1)]]


In [146]:
'''
Gensim creates a unique id for each word in the document.
The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the first document.

Likewise, word id 1 occurs twice and so on.

This is used as the input by the LDA model.

If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

id2word[0]
'addition'

Or, you can see a human-readable form of the corpus itself.

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('maryland_college', 1),
  (..truncated..)]]


Alright, without digressing further let’s jump back on track with the next step:

Building the topic model.
'''

"\nGensim creates a unique id for each word in the document.\nThe produced corpus shown above is a mapping of (word_id, word_frequency).\n\nFor example, (0, 1) above implies, word id 0 occurs once in the first document.\n\nLikewise, word id 1 occurs twice and so on.\n\nThis is used as the input by the LDA model.\n\nIf you want to see what word a given id corresponds to, pass the id as a key to the dictionary.\n\nid2word[0]\n'addition'\n\nOr, you can see a human-readable form of the corpus itself.\n\n# Human readable format of corpus (term-frequency)\n[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]\n\n[[('addition', 1),\n  ('anyone', 2),\n  ('body', 1),\n  ('bricklin', 1),\n  ('bring', 1),\n  ('call', 1),\n  ('car', 5),\n  ('could', 1),\n  ('day', 1),\n  ('door', 2),\n  ('early', 1),\n  ('engine', 1),\n  ('enlighten', 1),\n  ('front_bumper', 1),\n  ('maryland_college', 1),\n  (..truncated..)]]\n\n\nAlright, without digressing further let’s jump back on track with the nex

In [33]:
'''
We have everything required to train the LDA model.
In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics.
According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk.

update_every determines how often the model parameters should be updated and 
passes is the total number of training passes.
'''
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [34]:
'''
The above LDA model is built with 20 different topics where each topic is a 
combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of 
each keyword using lda_model.print_topics() as shown next.
'''
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

'''
The output will look like this...


[(0,
  '0.016*"car" + 0.014*"power" + 0.010*"light" + 0.009*"drive" + 0.007*"mount" '
  '+ 0.007*"controller" + 0.007*"cool" + 0.007*"engine" + 0.007*"back" + '
  '0.006*"turn"'),
 (1,
  '0.072*"line" + 0.066*"organization" + 0.037*"write" + 0.032*"article" + '
  '0.028*"university" + 0.027*"nntp_post" + 0.026*"host" + 0.016*"reply" + '
  '0.014*"get" + 0.013*"thank"'),
 (2,
  '0.017*"patient" + 0.011*"study" + 0.010*"slave" + 0.009*"wing" + '
  '0.009*"disease" + 0.008*"food" + 0.008*"eat" + 0.008*"pain" + '
  '0.007*"treatment" + 0.007*"syndrome"'),
 (3,
  '0.013*"key" + 0.009*"use" + 0.009*"may" + 0.007*"public" + 0.007*"system" + '
  '0.007*"order" + 0.007*"government" + 0.006*"state" + 0.006*"provide" + '
  '0.006*"law"'),
  
  so on...



How to interpret this?

Topic 0 is a represented as _

0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” 
+ 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” 
+ 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are:
‘car’, ‘power’, ‘light’.. and so on
and the weight of ‘car’ on topic 0 is 0.016.

--> The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be?
You may summarise it either are ‘cars’ or ‘automobiles’.

Likewise, can you go through the remaining topic keywords and judge what the topic is?
https://www.machinelearningplus.com/wp-content/uploads/2018/03/Inferring-Topic-from-Keywords.png
'''

[(0,
  '0.025*"much" + 0.019*"greyandsander" + 0.017*"dm" + 0.012*"time" + '
  '0.010*"look" + 0.010*"decor" + 0.010*"interior" + 0.009*"get" + '
  '0.009*"wood" + 0.009*"sofas"'),
 (1,
  '0.023*"italian" + 0.023*"price" + 0.023*"sofa" + 0.021*"grain" + '
  '0.020*"table" + 0.019*"jethro" + 0.018*"know" + 0.016*"leather" + '
  '0.015*"chair" + 0.012*"love"'),
 (2,
  '0.068*"homedecor" + 0.064*"homeinterior" + 0.061*"homegoal" + '
  '0.061*"sghomedecor" + 0.061*"homeinspo" + 0.058*"interiordesign" + '
  '0.041*"singapore" + 0.040*"instasg" + 0.039*"furnituredesign" + '
  '0.037*"greyandsander"')]


'\nThe output will look like this...\n\n\n[(0,\n  \'0.016*"car" + 0.014*"power" + 0.010*"light" + 0.009*"drive" + 0.007*"mount" \'\n  \'+ 0.007*"controller" + 0.007*"cool" + 0.007*"engine" + 0.007*"back" + \'\n  \'0.006*"turn"\'),\n (1,\n  \'0.072*"line" + 0.066*"organization" + 0.037*"write" + 0.032*"article" + \'\n  \'0.028*"university" + 0.027*"nntp_post" + 0.026*"host" + 0.016*"reply" + \'\n  \'0.014*"get" + 0.013*"thank"\'),\n (2,\n  \'0.017*"patient" + 0.011*"study" + 0.010*"slave" + 0.009*"wing" + \'\n  \'0.009*"disease" + 0.008*"food" + 0.008*"eat" + 0.008*"pain" + \'\n  \'0.007*"treatment" + 0.007*"syndrome"\'),\n (3,\n  \'0.013*"key" + 0.009*"use" + 0.009*"may" + 0.007*"public" + 0.007*"system" + \'\n  \'0.007*"order" + 0.007*"government" + 0.006*"state" + 0.006*"provide" + \'\n  \'0.006*"law"\'),\n  \n  so on...\n\n\n\nHow to interpret this?\n\nTopic 0 is a represented as _\n\n0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” \n+ 0.007“mount” + 0.007“controller” + 0.00

In [35]:
'''
Model perplexity and topic coherence provide a convenient measure to judge 
how good a given topic model is. In my experience, topic coherence score, 
in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

'''
Perplexity:  -8.86067503009

Coherence Score:  0.532947587081


There you have a coherence score of 0.53.
'''


Perplexity:  -4.516973794531733

Coherence Score:  0.7746888618420756


'\nPerplexity:  -8.86067503009\n\nCoherence Score:  0.532947587081\n\n\nThere you have a coherence score of 0.53.\n'

In [36]:
'''
Now that the LDA model is built, the next step is to examine the produced 
topics and the associated keywords. There is no better tool than pyLDAvis package’s 
interactive chart and is designed to work well with jupyter notebooks.
'''
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.328034  0.002497       1        1  71.845750
0     -0.159034 -0.124537       2        1  15.430356
1     -0.169000  0.122040       3        1  12.723894, topic_info=              Term        Freq       Total Category  logprob  loglift
15       homedecor  118.000000  118.000000  Default  30.0000  30.0000
156   homeinterior  111.000000  111.000000  Default  29.0000  29.0000
216       homegoal  106.000000  106.000000  Default  28.0000  28.0000
219    sghomedecor  106.000000  106.000000  Default  27.0000  27.0000
217      homeinspo  106.000000  106.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
252             pm    2.295326    3.830889   Topic3  -4.8961   1.5495
46             let    1.604592    3.127408   Topic3  -5.2541   1.3944
33            take    1.415212    2.950754   Topic3  -5.3797   1.3269
37            look    1.604562    5.665424   Topic3  -5.2541   0.8002
14   greyandsander    2.775214   74.622489   Topic3  -4.7062  -1.2300

[130 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
360       2  0.715633         artwork
422       2  0.715716         awesome
171       2  0.878734       beautiful
169       2  0.814011           bench
160       1  0.990925     blackwalnut
...     ...       ...             ...
229       2  0.878730            well
36        2  0.790482            wood
417       1  0.948901       woodchair
106       1  0.980664  woodslabstable
106       2  0.024517  woodslabstable

[108 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [151]:
'''
So how to infer pyLDAvis’s output?

Each bubble on the left-hand side plot represents a topic. 
The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered 
throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, 
small sized bubbles clustered in one region of the chart.

Alright, if you move the cursor over one of the bubbles, 
the words and bars on the right-hand side will update. 
These words are the salient keywords that form the selected topic.

We have successfully built a good looking topic model.
'''

'\nSo how to infer pyLDAvis’s output?\n\nEach bubble on the left-hand side plot represents a topic. \nThe larger the bubble, the more prevalent is that topic.\n\nA good topic model will have fairly big, non-overlapping bubbles scattered \nthroughout the chart instead of being clustered in one quadrant.\n\nA model with too many topics, will typically have many overlaps, \nsmall sized bubbles clustered in one region of the chart.\n\nAlright, if you move the cursor over one of the bubbles, \nthe words and bars on the right-hand side will update. \nThese words are the salient keywords that form the selected topic.\n\nWe have successfully built a good looking topic model.\n'

In [37]:
pyLDAvis.save_html(vis, 'greyandsanders_lda.html')